# DATASET

#### Source : https://www.kaggle.com/san-francisco/sf-building-permits-and-contacts

#### Data includes application/permit numbers, contact name, address, and license numbers. Data is uploaded weekly by DBI. This data set is intended to be joined to the Building Permits data set.

# Initialization

In [1]:
# Import findspark to read SPARK_HOME and HADOOP_HOME

import findspark
findspark.init()

In [2]:
# Import required library

from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [3]:
# Print Spark object ID
print(spark)

# Loading Data

In [4]:
# Import Dataset Business Licenses & Owners
df = spark.read.csv("C://Users//RPL-ICHAL//Documents//Bonbon//building-permits.csv", header=True, inferSchema=True)
df2 = spark.read.csv("C://Users//RPL-ICHAL//Documents//Bonbon//building-permits-contacts.csv", header=True, inferSchema=True)

In [5]:
df.count()

1091782

In [6]:
df2.count()

638155

In [7]:
df.printSchema()

root
 |-- Permit Number: string (nullable = true)
 |-- Permit Type: integer (nullable = true)
 |-- Permit Type Definition: string (nullable = true)
 |-- Permit Creation Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- Lot: string (nullable = true)
 |-- Street Number: integer (nullable = true)
 |-- Street Number Suffix: string (nullable = true)
 |-- Street Name: string (nullable = true)
 |-- Street Suffix: string (nullable = true)
 |-- Unit: integer (nullable = true)
 |-- Unit Suffix: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Current Status: string (nullable = true)
 |-- Current Status Date: string (nullable = true)
 |-- Filed Date: string (nullable = true)
 |-- Issued Date: string (nullable = true)
 |-- Completed Date: string (nullable = true)
 |-- First Construction Document Date: string (nullable = true)
 |-- Structural Notification: string (nullable = true)
 |-- Number of Existing Stories: string (nullable = true)
 |-- Number

In [8]:
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Permit Number: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Role: string (nullable = true)
 |-- Agent Address: string (nullable = true)
 |-- Agent Address2: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Agent Zipcode: string (nullable = true)
 |-- Firm Name: string (nullable = true)
 |-- Firm Address: string (nullable = true)
 |-- Firm City: string (nullable = true)
 |-- Firm State: string (nullable = true)
 |-- Firm Zipcode: string (nullable = true)
 |-- PTS Agent ID: string (nullable = true)
 |-- From Date: string (nullable = true)
 |-- To Date: long (nullable = true)
 |-- License1: string (nullable = true)
 |-- License2: string (nullable = true)



In [9]:
# Show Data Licenses
df.show()

+-------------+-----------+----------------------+--------------------+-----+----+-------------+--------------------+--------------+-------------+----+-----------+--------------------+--------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+-----------------------+--------------------------+--------------------------+-----------------------------+----------------+----------------------+--------------+------------+-----------------+--------------+-----------------+--------------+--------+---------------+--------------------------+--------------------------------------+--------------------------+--------------------------------------+-----------+-------------------+-----------------------------------+-------+--------------------+-------------+
|Permit Number|Permit Type|Permit Type Definition|Permit Creation Date|Block| Lot|Street Number|Street Number Suffix|   Street Name|Street Suffix|Unit|Unit Suffix|         Descr

In [10]:
# Show Data Licenses
df2.show()

+------+-------------+--------------------+---------+----------+-------------+--------------+----+-----+-------------+--------------------+--------------------+----------------+----------+------------+------------+---------+-------+--------+--------+
|    ID|Permit Number|          First Name|Last Name|      Role|Agent Address|Agent Address2|City|State|Agent Zipcode|           Firm Name|        Firm Address|       Firm City|Firm State|Firm Zipcode|PTS Agent ID|From Date|To Date|License1|License2|
+------+-------------+--------------------+---------+----------+-------------+--------------+----+-----+-------------+--------------------+--------------------+----------------+----------+------------+------------+---------+-------+--------+--------+
|318704|      9801804|       James (Larry)|     Pace|contractor|         null|          null|null| null|         null|Cannon Constructo...|       301 Howard St|   San Francisco|        CA|  94105-0000|      699760|886464000|   null|  918396|    nu

In [11]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("permit")
df2.createOrReplaceTempView("contact")

# Data Mining

In [22]:
# 1. Show Building owner, firm name, and permit type
query1 = spark.sql("SELECT CONCAT(`First Name`,' ',`Last Name`) AS FullName,`Firm Name`, `Permit Type Definition` \
                    FROM permit JOIN contact ON permit.`Permit Number`=contact.`Permit Number`")

In [23]:
query1.show()

+--------------------+--------------------+----------------------+
|            FullName|           Firm Name|Permit Type Definition|
+--------------------+--------------------+----------------------+
| Christopher Stanton|Paragon General C...|  otc alterations p...|
|        Doug Gearman|Turner Constructi...|  otc alterations p...|
|                null| Cascade Roofing Co.|  otc alterations p...|
|        David Grassi|Professional Conc...|  otc alterations p...|
|                null|Trans -City Const...|  otc alterations p...|
|                null|Corporate Builder...|  additions alterat...|
| Steve Sung Kun Hong|  Westech Roofing Co|  otc alterations p...|
|   Jaidin Consulting|               Group|  additions alterat...|
|                null|Mitchell Engineer...|  otc alterations p...|
|           Tom Ledda|  Ledda Plumbing Inc|  otc alterations p...|
|            Tung Luu|All Four Season's...|  otc alterations p...|
|                null|David Silverberg ...|  otc alterations p

In [28]:
# 2. Total building permit which is already expired based on the permit type
query2 = spark.sql("SELECT `Permit Type Definition`, COUNT(`Current Status`) as TOTAL \
                    FROM permit \
                    WHERE `Current Status`='expired'\
                    GROUP BY `Permit Type Definition` \
                    ORDER BY Total DESC")

In [29]:
query2.show()

+----------------------+------+
|Permit Type Definition| TOTAL|
+----------------------+------+
|  otc alterations p...|151470|
|  additions alterat...| 65598|
|          sign - erect|  4136|
|  new construction ...|  1508|
|           demolitions|   590|
|  wall or painted sign|   379|
|      new construction|   324|
|  grade or quarry o...|   239|
+----------------------+------+



In [41]:
# 3. New construction building in 2019
query3 = spark.sql("SELECT `Firm Name`, TO_DATE(`Permit Creation Date`, 'MM/DD/YYYY') AS Date \
                    FROM permit JOIN contact ON permit.`Permit Number`=contact.`Permit Number` \
                    WHERE `Permit Type Definition`='new construction' AND YEAR(`Permit Creation Date`)='2019'")

In [42]:
query3.show()

+--------------------+----------+
|           Firm Name|      Date|
+--------------------+----------+
|                null|2019-02-11|
|Cahill Contractor...|2019-02-11|
|Gary Bell & Assoc...|2019-01-30|
|Christiani Johnso...|2019-01-30|
|            Doug Lee|2019-01-04|
|        William Chan|2019-01-04|
+--------------------+----------+



In [64]:
#4. Gap between estimated and revised cost of building permition
query4 = spark.sql("SELECT `Firm Name`, `Permit Type Definition`, `Estimated Cost`, `Revised Cost`, ABS(`Estimated Cost`-`Revised Cost`) As Gap \
                    FROM permit JOIN contact ON permit.`Permit Number`=contact.`Permit Number`")

In [65]:
query4.show()

+--------------------+----------------------+--------------+------------+------+
|           Firm Name|Permit Type Definition|Estimated Cost|Revised Cost|   Gap|
+--------------------+----------------------+--------------+------------+------+
|Paragon General C...|  otc alterations p...|         20000|       20000|   0.0|
|Turner Constructi...|  otc alterations p...|         10000|       10000|   0.0|
| Cascade Roofing Co.|  otc alterations p...|          null|        3800|  null|
|Professional Conc...|  otc alterations p...|          5500|        5500|   0.0|
|Trans -City Const...|  otc alterations p...|          null|        5000|  null|
|Corporate Builder...|  additions alterat...|      15000000|     1500000|1.35E7|
|  Westech Roofing Co|  otc alterations p...|          null|        4200|  null|
|               Group|  additions alterat...|        150000|      150000|   0.0|
|Mitchell Engineer...|  otc alterations p...|          8000|        8000|   0.0|
|  Ledda Plumbing Inc|  otc 

In [69]:
#5. Building existing used as family dwelling
query5 = spark.sql("SELECT `Firm Name`, `Existing Use` \
                    FROM permit JOIN contact ON permit.`Permit Number`=contact.`Permit Number` \
                    WHERE `Existing Use` LIKE '%family dwelling%'")

In [70]:
query5.show()

+--------------------+-----------------+
|           Firm Name|     Existing Use|
+--------------------+-----------------+
| Cascade Roofing Co.|2 family dwelling|
|Professional Conc...|1 family dwelling|
|Trans -City Const...|1 family dwelling|
|  Westech Roofing Co|2 family dwelling|
|               Group|1 family dwelling|
|All Four Season's...|2 family dwelling|
|David Silverberg ...|1 family dwelling|
|   Delta Roofing Inc|1 family dwelling|
|Goffo Constructio...|1 family dwelling|
|C & J Roofing Com...|1 family dwelling|
|   Tony Construction|1 family dwelling|
|                null|2 family dwelling|
|                null|2 family dwelling|
|Shaughnessy Roofi...|1 family dwelling|
|      Daniel Ehrmann|2 family dwelling|
|Mccarthy /Fred Ha...|1 family dwelling|
|A B C  Window Con...|1 family dwelling|
|   Kelly Roofing Inc|1 family dwelling|
| Tom Lee Roofing Inc|2 family dwelling|
|         101 Roofing|1 family dwelling|
+--------------------+-----------------+
only showing top

In [71]:
#6. Building permition owned by agent
query6 = spark.sql("SELECT `Firm Name`, Role\
                    FROM contact \
                    WHERE Role LIKE '%agent%'")

In [72]:
query6.show()

+--------------------+--------------------+
|           Firm Name|                Role|
+--------------------+--------------------+
|Kotas/pontaleoni ...|authorized agent-...|
|Kotas/pontaleoni ...|authorized agent-...|
|                null|authorized agent-...|
|         Carolyn Liu|authorized agent-...|
|                null|authorized agent-...|
|Rhl Design Group ...|authorized agent-...|
|                null|authorized agent-...|
|    Jadin Consulting|authorized agent-...|
|                null|authorized agent-...|
|           Gci, Inc.|authorized agent-...|
|Permit Drafting S...|authorized agent-...|
|               Terry|authorized agent-...|
|J Fitzimmons Buil...|authorized agent-...|
|                null|authorized agent-...|
|                null|authorized agent-...|
| Permit Services Inc|authorized agent-...|
| Natoma Architec Inc|authorized agent-...|
|                 Llc|authorized agent-...|
|Red Hawk Fire & S...|authorized agent-...|
|                null|authorized